In [44]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [45]:
randomlist =[]

for i in range(0,10):
    randomlist.append(random.randint(0,3999))

### Διαβασμα Εικονων και μετατροπη σε grayscale

In [46]:
total_imgs = dict()
for item in randomlist:
    temp = []
    nm_of_imgs=0
    for filename in glob.glob("train_data/"+str(item)+"/*.jpg"):
        img = mpimg.imread(filename)
        gray = np.dot(img[...,:3],[0.2989, 0.5870, 0.1140]) # conversion to grayscale
        gray = np.reshape(gray, 4096)
        temp.append(gray)
        nm_of_imgs += 1
        if nm_of_imgs==50:
            break

    total_imgs[item] = temp

In [47]:
folders = []
for folder in total_imgs:
    images_of_folder = []
    for img in total_imgs[folder]:
        folders.append(img)

In [48]:
all_photos = np.array(folders)

# Dimensionality Reduction Using PCA

In [49]:
pca = PCA(n_components=100)
pca_100 = pca.fit_transform(all_photos)

pca = PCA(n_components=50)
pca_50 = pca.fit_transform(all_photos)

pca = PCA(n_components=25)
pca_25 = pca.fit_transform(all_photos)

## Autoencoder με Aρχιτεκτονική d – d/4 - M – d/4 – d

Βλέπε αρχείο autoencoder.py

# Classification using K - Means

In [50]:
def cal_for_purity(row):
    counter_matrix = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in row:
        if(i>= 0 and i < 50):
            counter_matrix[0] = counter_matrix[0] + 1
        elif (i>= 50 and i < 100):
            counter_matrix[1] = counter_matrix[1] + 1
        elif (i>= 100 and i < 150):
            counter_matrix[2] = counter_matrix[2] + 1
        elif (i>= 150 and i < 200):
            counter_matrix[3] = counter_matrix[3] + 1
        elif (i>= 200 and i < 250):
            counter_matrix[4] = counter_matrix[4] + 1
        elif (i>= 250 and i < 300):
            counter_matrix[5] = counter_matrix[5] + 1
        elif (i>= 300 and i < 350):
            counter_matrix[6] = counter_matrix[6] + 1
        elif (i>= 350 and i < 400):
            counter_matrix[7] = counter_matrix[7] + 1
        elif (i>= 400 and i < 450):
            counter_matrix[8] = counter_matrix[8] + 1
        else:
            counter_matrix[9] = counter_matrix[9] + 1
    return max(counter_matrix)

In [51]:
def metrics(predictions):
    purity = 0
    total_fa = 0
    for i in range(10):

        row = np.where(predictions==i)[0]  # row in predictions for elements of cluster i
        num = row.shape[0]       #  number of elements for each cluster
        r = np.floor(num/10.)    # number of rows in the figure of the cluster 

        major_cat = cal_for_purity(row)
        purity = purity + major_cat
        true_positives = major_cat
        false_postives = num - true_positives
        false_negatives = 50 - true_positives
        precision = true_positives / (true_positives + false_postives)
        recall = true_positives / (true_positives + false_negatives)
        a = 1
        fa = (1 + a) / ((1 / precision) + (a / recall))
        total_fa = total_fa + fa

    print("Purity = " + str(purity/500))
    print("Fa = " + str(total_fa))

In [52]:
kmeans = KMeans(n_clusters=10).fit(pca_100)
predictions = kmeans.predict(pca_100)
metrics(predictions)

Purity = 0.272
Fa = 2.727878098551799


In [53]:
kmeans = KMeans(n_clusters=10).fit(pca_50)
predictions = kmeans.predict(pca_50)
metrics(predictions)

Purity = 0.284
Fa = 2.8189048722757777


In [54]:
kmeans = KMeans(n_clusters=10).fit(pca_25)
predictions = kmeans.predict(pca_25)
metrics(predictions)

Purity = 0.254
Fa = 2.566510698254261


# Agglomerative Hierarchical Clustering 

In [55]:
predictions = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward').fit_predict(pca_100)
metrics(predictions)

Purity = 0.266
Fa = 2.590245155205283


In [56]:
predictions = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward').fit_predict(pca_50)
metrics(predictions)

Purity = 0.254
Fa = 2.474659295177248


In [57]:
predictions = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward').fit_predict(pca_25)
metrics(predictions)

Purity = 0.264
Fa = 2.5279050204670668
